This file builds upon <b>MongoDB & MapReduce.ipynb</b>.  
<br>In the last file, I developed a code that reads and uploads part of “reviews_electronics.16.json” to a MongoDB database called “amazon”.

This time we will augment the code to create a collection called “electronics” in “amazon” that contains each line of “reviews_electronics.16.json” as an individual document.

I will write code that reads all documents in “electronics”, and prints the first 25 documents to screen.

In the second part, we will create a SQL database.

In [ ]:
from pymongo import MongoClient
from pprint import pprint
import json
import sqlalchemy as db
import pandas as pd

## a) Create a collection called “electronics” in “amazon” that contains each line of “reviews_electronics.16.json” as an individual document. Write code that reads all documents in “electronics”, and prints the first 25 documents to screen.

In [2]:
#Connects to the mongodb running on localhost at port 27017
connection = MongoClient('localhost', 27017)
#Creates a database names amazon
mongodb = connection.amazon

In [3]:
#Reads the file data
with open("reviews_electronics.16.json", "r") as file:
    data = file.readlines()

In [4]:
#Loops over the data and inserts each line as a document to the database
for i in range(len(data)):
    mongodb.electronics.insert_one(json.loads(data[i]))

In [5]:
#Query the first 25 documents from the table and print it on screen
results = mongodb.electronics.find().limit(25)
for each in results:
    pprint(each)

{'_id': ObjectId('5e5b47997a50773996690329'),
 'asin': '0132793040',
 'helpful': [1, 1],
 'overall': 5.0,
 'reviewText': 'Corey Barker does a great job of explaining Blend Modes in '
               'this DVD. All of the Kelby training videos are great but '
               'pricey to buy individually. If you really want bang for your '
               'buck just subscribe to Kelby Training online.',
 'reviewTime': '04 13, 2013',
 'reviewerID': 'AKM1MP6P0OYPR',
 'reviewerName': 'Vicki Gibson "momo4"',
 'summary': 'Very thorough',
 'unixReviewTime': 1365811200}
{'_id': ObjectId('5e5b47997a5077399669032a'),
 'asin': 'B00E4KP4W6',
 'helpful': [1, 1],
 'overall': 5.0,
 'reviewText': "I can't believe I waited to long to switch to a glass screen "
               'protector.  I love this.  It feels and looks like there is no '
               'protector on.  It does show fingerprints, which I think is '
               'inevitable unless you use a matte finish screen protector, but '
             

## b) Create a SQL database named “amazon_SQL”, In this database, create a SQL table named “electronics_SQL” that can hold each document from “electronics”.  Develop code (in Python or Java) that (1) reads all documents from “electronics” in MongoDB, and (2) writes that content to “electronics_SQL” in SQL. Print the first 25 documents to the screen. 

 

In [6]:
# using create_engine function from sqlalchemy to connect to the mysql
user="root"
passwd = ""
host = "localhost"
port = "3306"
database_name = "amazon_sql"
mysql_engine = db.create_engine('mysql+mysqlconnector://{0}:{1}@{2}:{3}/'.format(user, passwd, host, port) + "?use_pure=True")
# creates the database names amazon_sql
mysql_engine.execute("CREATE DATABASE IF NOT EXISTS {0} ".format(database_name))

In [7]:
# creates the connection to the amazon_sql database
mysql_engine = db.create_engine('mysql+mysqlconnector://{0}:{1}@{2}:{3}/'.format(user, passwd, host, port) + database_name + "?use_pure=True")

In [8]:
# creates the string with the table create statement
table_name = "electronics_sql"
create_statement = "CREATE TABLE " + table_name + "(id INT NOT NULL AUTO_INCREMENT, reviewerID VARCHAR(15), asin CHAR(10), reviewerName VARCHAR(40), helpful_votes INT, helpful_total INT, overall INT, summary VARCHAR(100), unixReviewTime INT UNSIGNED, reviewTime Date, PRIMARY KEY(id))"
print(create_statement)

CREATE TABLE electronics_sql(id INT NOT NULL AUTO_INCREMENT, reviewerID VARCHAR(15), asin CHAR(10), reviewerName VARCHAR(40), helpful_votes INT, helpful_total INT, overall INT, summary VARCHAR(100), unixReviewTime INT UNSIGNED, reviewTime Date, PRIMARY KEY(id))


In [9]:
# Executes the create table statement
mysql_engine.execute(create_statement)

In [10]:
# Fetches all the documents from the electronics collection
results = mongodb.electronics.find()

In [11]:
# Converts the results to a dataframe
data = pd.DataFrame(list(results))

In [12]:
# Converts the reviewTime column from string to date format
data['reviewTime'] =  pd.to_datetime(data["reviewTime"], format='%m %d, %Y')

In [13]:
# extracts the first value from the helpful list
data["helpful1"] = data["helpful"].apply(lambda x: x[0])

In [14]:
# extracts the second value from the helpful list
data["helpful2"] = data["helpful"].apply(lambda x: x[1])

In [15]:
# rearranges the column order to match the sql table schema
data = data[["reviewerID", "asin", "reviewerName", "helpful1", "helpful2", "overall", "summary", "unixReviewTime", "reviewTime"]]

In [16]:
# rename the columns helpful1 to helpful_votes and helpful2 to helpful_total
data.columns = ["reviewerID", "asin", "reviewerName", "helpful_votes", "helpful_total", "overall", "summary", "unixReviewTime", "reviewTime"]

In [17]:
data.head()

,reviewerID,asin,reviewerName,helpful_votes,helpful_total,overall,summary,unixReviewTime,reviewTime
0,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""",1,1,5.0,Very thorough,1365811200,2013-04-13
1,A2X8VX4DPMQFQQ,B00E4KP4W6,lily68,1,1,5.0,LOVE this screen protector!!,1393459200,2014-02-27
2,A1JEKX4D58576X,B00E4KP4W6,Mara Milagros,2,2,5.0,excellent,1394496000,2014-03-11
3,ATM6RRB90D8DV,B00E4KP4W6,"Matthew D. Irmen ""Jazz Fan""",1,2,4.0,"Nice looking, easy to install glass screen pro...",1393891200,2014-03-04
4,A3QOYJ97A6OMX7,B00E4KP4W6,pharmdaddyd,1,1,5.0,great for the price,1388361600,2013-12-30


In [18]:
# Writes the data to the mysql table
data.to_sql(table_name, con = mysql_engine, if_exists = 'replace', chunksize = 100, index = False)